In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np

# Explanation and walkthrough

- Number of users after cleaning: 359,788
- Number of books after cleaning: 452,982
- Number of reviews after cleaning: 6,555,799

# Clean Book Data, Load Author Data + Genres

In [ ]:
df = pd.read_json('/content/drive/MyDrive/Recommender Systems/GroupProject/Books/goodreads_books_children.json', lines=True)
df.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,1599150603,7,[],US,,"[{'count': '56', 'name': 'to-read'}, {'count':...",,false,4.13,B00DU10PUG,...,9,,2006,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,46,278578,The Aeneid for Boys and Girls,The Aeneid for Boys and Girls
1,1934876569,6,[151854],US,,"[{'count': '515', 'name': 'to-read'}, {'count'...",,false,4.22,,...,3,,2009,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,6066812,98,701117,All's Fairy in Love and War (Avalon: Web of Ma...,All's Fairy in Love and War (Avalon: Web of Ma...
2,0590417010,193,[],US,eng,"[{'count': '450', 'name': 'to-read'}, {'count'...",,false,4.43,B017RORXNI,...,9,,1995,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,1331,86259,Dog Heaven,Dog Heaven
3,0915190575,4,[],US,,"[{'count': '8', 'name': 'to-read'}, {'count': ...",,false,4.29,,...,,,,https://www.goodreads.com/book/show/3209312-mo...,https://s.gr-assets.com/assets/nophoto/book/11...,3209312,11,3242879,"Moths and Mothers, Feathers and Fathers: A Sto...","Moths and Mothers, Feathers and Fathers: A Sto..."
4,1416904999,4,[],US,,"[{'count': '8', 'name': 'to-read'}, {'count': ...",,false,3.57,,...,6,,2005,https://www.goodreads.com/book/show/1698376.Wh...,https://s.gr-assets.com/assets/nophoto/book/11...,1698376,23,1695373,What Do You Do?,What Do You Do?


In [5]:
def clean_books_df(file_path):
    df = pd.read_json(file_path, lines=True)

    columns_to_drop = [
        'kindle_asin', 'popular_shelves', 'asin', 'is_ebook', 'isbn13',
        'edition_information', 'url', 'image_url', 'title_without_series',
        'language_code', 'similar_books', 'format', 'link', 'work_id',
        'text_reviews_count', 'ratings_count'
    ]
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], axis=1, inplace=True)

    nona_columns = [
        'book_id', 'isbn', 'title', 'average_rating', 'authors', 'publisher',
        'description', 'country_code', 'num_pages', 'publication_day',
        'publication_month', 'publication_year'
    ]
    df[nona_columns] = df[nona_columns].replace('', np.nan)
    df.dropna(subset=nona_columns, inplace=True)

    df['publication_year'] = df['publication_year'].astype(str)
    df = df[df['publication_year'].str.len() == 4]

    df['publication_day'] = df['publication_day'].apply(lambda x: f"{int(x):02d}" if pd.notnull(x) else x)
    df['publication_month'] = df['publication_month'].apply(lambda x: f"{int(x):02d}" if pd.notnull(x) else x)

    df['publication_date'] = df['publication_day'] + '-' + df['publication_month'] + '-' + df['publication_year']
    df['publication_date'] = pd.to_datetime(df['publication_date'], format='%d-%m-%Y', errors='coerce')

    df.dropna(subset=['publication_date'], inplace=True)

    desired_column_order = [
        'book_id', 'isbn', 'title', 'average_rating', 'authors', 'publisher',
        'description', 'series', 'country_code', 'num_pages', 'publication_date'
    ]
    df = df[desired_column_order]

    return df

In [13]:
book_children_output_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/CleanedData/book_children.csv'
book_children_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/goodreads_books_children.json'
book_children = clean_books_df(book_children_path)
book_children.to_csv(book_children_output_path, index=False)
book_children.head()

,book_id,isbn,title,average_rating,authors,publisher,description,series,country_code,num_pages,publication_date
0,287141,1599150603,The Aeneid for Boys and Girls,4.13,"[{'author_id': '3041852', 'role': ''}]",Yesterday's Classics,"Relates in vigorous prose the tale of Aeneas, ...",[],US,162,2006-09-13
1,6066812,1934876569,All's Fairy in Love and War (Avalon: Web of Ma...,4.22,"[{'author_id': '19158', 'role': ''}]",Seven Seas,"To Kara's astonishment, she discovers that a p...",[151854],US,216,2009-03-03
2,89378,0590417010,Dog Heaven,4.43,"[{'author_id': '5411', 'role': ''}]",Blue Sky Press,In Newbery Medalist Cynthia Rylant's classic b...,[],US,40,1995-09-01
4,1698376,1416904999,What Do You Do?,3.57,"[{'author_id': '169159', 'role': ''}]",Little Simon,WHAT DO YOU DO?\nA hen lays eggs...\nA cow giv...,[],US,24,2005-06-01
7,3631900,0884482987,Amadi's Snowman: A Story of Reading,3.89,"[{'author_id': '1540277', 'role': ''}, {'autho...",Tilbury House Publishers,When Amadi disobeys his mother and runs off to...,[],US,32,2008-05-01


In [6]:
book_fantasy_output_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/CleanedData/book_fantasy.csv'
book_fantasy_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/goodreads_books_fantasy_paranormal.json'
book_fantasy = clean_books_df(book_fantasy_path)
book_fantasy.to_csv(book_fantasy_output_path, index=False)
book_fantasy.head()

,book_id,isbn,title,average_rating,authors,publisher,description,series,country_code,num_pages,publication_date
1,6066812,1934876569,All's Fairy in Love and War (Avalon: Web of Ma...,4.22,"[{'author_id': '19158', 'role': ''}]",Seven Seas,"To Kara's astonishment, she discovers that a p...",[151854],US,216,2009-03-03
8,21401181,0698143760,"Half Bad (Half Life, #1)",3.80,"[{'author_id': '7314532', 'role': ''}]",Viking Children's,Wanted by no one.\nHunted by everyone.\nSixtee...,[493993],US,416,2014-03-04
12,780917,0765313286,"The Serpent and the Rose (War of the Rose, #1)",3.49,"[{'author_id': '410348', 'role': ''}]",Tor Books,The beautiful Averil is heir to the Duchy of Q...,[152365],US,320,2007-03-06
16,16693030,1406346950,A Monster Calls,4.37,"[{'author_id': '370361', 'role': ''}]",Walker Childrens Hardbacks,"An unflinching, darkly funny, and deeply movin...",[],US,240,2012-11-01
23,18656391,0316219169,Baptism of Fire,4.36,"[{'author_id': '38569', 'role': ''}]",Orbit,The Wizards Guild has been shattered by a coup...,[159489],US,288,2014-06-24


In [7]:
book_history_output_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/CleanedData/book_history.csv'
book_history_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/goodreads_books_history_biography.json'
book_history = clean_books_df(book_history_path)
book_history.to_csv(book_history_output_path, index=False)
book_history.head()

,book_id,isbn,title,average_rating,authors,publisher,description,series,country_code,num_pages,publication_date
0,287141,1599150603,The Aeneid for Boys and Girls,4.13,"[{'author_id': '3041852', 'role': ''}]",Yesterday's Classics,"Relates in vigorous prose the tale of Aeneas, ...",[],US,162,2006-09-13
1,6066814,184737297X,"Crowner Royal (Crowner John Mystery, #13)",3.93,"[{'author_id': '37778', 'role': ''}]",Simon & Schuster UK,"London, 1196. At the command of Richard the Li...",[169353],US,400,2009-04-06
2,89377,037583687X,Penny from Heaven,3.98,"[{'author_id': '137561', 'role': ''}]",Random House Books for Young Readers,It's 1953 and 11-year-old Penny dreams of a su...,[],US,288,2006-07-25
4,18628480,8864116435,Stoner,4.28,"[{'author_id': '51229', 'role': ''}, {'author_...",Fazi,Stoner e il racconto della vita di un uomo tra...,[],US,332,2012-05-28
7,13707894,0393929876,Eighteenth-Century Europe: Tradition and Progr...,3.57,"[{'author_id': '8070', 'role': ''}, {'author_i...",W. W. Norton & Company,The new second edition of the standard text on...,[414880],US,362,2012-01-09


In [9]:
book_mistery_output_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/CleanedData/book_mistery.csv'
book_mistery_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/goodreads_books_mystery_thriller_crime.json'
book_mistery = clean_books_df(book_mistery_path)
book_mistery.to_csv(book_mistery_output_path, index=False)
book_mistery.head()

,book_id,isbn,title,average_rating,authors,publisher,description,series,country_code,num_pages,publication_date
0,6066814,184737297X,"Crowner Royal (Crowner John Mystery, #13)",3.93,"[{'author_id': '37778', 'role': ''}]",Simon & Schuster UK,"London, 1196. At the command of Richard the Li...",[169353],US,400,2009-04-06
7,2805495,0752844458,Wycliffe and the Cycle of Death,3.61,"[{'author_id': '1533165', 'role': ''}]",Orion,A respectable bookseller is found bludgeoned a...,[326237],US,320,2001-08-02
8,22722787,8293326247,The Cost of Doing Business,4.14,"[{'author_id': '4577517', 'role': ''}]",280 Steps,"""Poetic, down trodden and nihilistic, Jonathan...",[],US,228,2014-11-04
10,17368265,1619694611,"White Fire (Pendergast, #13)",4.07,"[{'author_id': '12577', 'role': ''}, {'author_...",Grand Central Publishing,Past and present collide as Special Agent Pend...,[510553],US,13,2013-11-12
11,778285,0345367650,"Polar Star (Arkady Renko, #2)",3.99,"[{'author_id': '8258', 'role': ''}]",Ballantine Books,"In the long-awaited sequel to Gorky Park, Arka...",[163090],US,366,1990-06-13


In [10]:
book_poetry_output_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/CleanedData/book_poetry.csv'
book_poetry_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/goodreads_books_poetry.json'
book_poetry = clean_books_df(book_poetry_path)
book_poetry.to_csv(book_poetry_output_path, index=False)
book_poetry.head()

,book_id,isbn,title,average_rating,authors,publisher,description,series,country_code,num_pages,publication_date
1,22466716,0811223981,Fairy Tales: Dramolettes,3.83,"[{'author_id': '16073', 'role': ''}, {'author_...",New Directions,Fairy Tales gathers the unconventional verse d...,[],US,128,2015-04-20
3,926667,0156182890,The Cocktail Party,3.71,"[{'author_id': '18540', 'role': ''}]",Mariner Books,A modern verse play about the search for meani...,[],US,190,1964-03-18
4,29065952,1942004192,Louder Than Everything You Love,5.00,"[{'author_id': '14308759', 'role': ''}]",ELJ Publications,Louder Than Everything You Love is about trans...,[],US,118,2015-12-23
10,24849837,0692265295,Naked Soul: The Erotic Love Poems,3.95,"[{'author_id': '13260036', 'role': ''}]",Naked Soul Press,"""Erotic poetry that evokes feelings of joy, ha...",[],US,172,2015-01-09
12,20544060,1927575249,Black Liquor: Poems,4.00,"[{'author_id': '547708', 'role': ''}]",Caitlin Press,Dennis E. Bolen's forthcoming volume of poetry...,[],US,128,2013-09-01


In [11]:
book_romance_output_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/CleanedData/book_romance.csv'
book_romance_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/goodreads_books_romance.json'
book_romance = clean_books_df(book_romance_path)
book_romance.to_csv(book_poetry_output_path, index=False)
book_romance.head()

,book_id,isbn,title,average_rating,authors,publisher,description,series,country_code,num_pages,publication_date
2,3209316,1597371289,Emma,3.99,"[{'author_id': '1265', 'role': ''}]",Brilliance Audio,The funny and heartwarming story of a young la...,[],US,544,2005-09-25
12,18283095,4091838758,ノ・ゾ・キ・ア・ナ 8 [Nozoki Ana 8],3.71,"[{'author_id': '7066263', 'role': ''}, {'autho...",Xiao Xue Guan,"Si kiXue noCun Zai ga, tsuinimochidukinibarete...",[529990],US,188,2011-06-30
14,22499086,1627988912,Love Means... Patience,4.11,"[{'author_id': '2953781', 'role': ''}]",Dreamspinner Press,A Love Means... Story\nYears after his dischar...,[629995],US,200,2014-05-23
15,2250580,1933617640,"A.I. Revolution, Vol. 1",3.44,"[{'author_id': '1015982', 'role': ''}]",Go! Comi,"Like everyone else in the future, Sui's used t...",[910384],US,206,2007-01-02
21,24614930,1921901993,The Falls,4.08,"[{'author_id': '5137697', 'role': ''}]",Michael Joseph / Penguin Australia,"For as long as she can remember, Teagan Bliss ...",[],US,402,2015-04-22


In [12]:
book_ya_output_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/CleanedData/book_youngadult.csv'
book_ya_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/goodreads_books_young_adult.json'
book_ya = clean_books_df(book_ya_path)
book_ya.to_csv(book_poetry_output_path, index=False)
book_ya.head()

,book_id,isbn,title,average_rating,authors,publisher,description,series,country_code,num_pages,publication_date
2,21401181,0698143760,"Half Bad (Half Life, #1)",3.80,"[{'author_id': '7314532', 'role': ''}]",Viking Children's,Wanted by no one.\nHunted by everyone.\nSixtee...,[493993],US,416,2014-03-04
4,22642971,0990662616,The Body Electric,3.71,"[{'author_id': '4018722', 'role': ''}]",Scripturient Books,The future world is at peace.\nElla Shepherd h...,[],US,351,2014-10-06
5,31556136,0062373374,Like Water,3.89,"[{'author_id': '6537142', 'role': ''}]",Balzer + Bray,A gorgeously written and deeply felt literary ...,[],US,304,2017-10-17
9,25421507,1941103863,The Ticket,3.82,"[{'author_id': '539783', 'role': ''}]",Firefly Southern Fiction,Librarian Note: See Alternate Cover Edition ....,[],US,210,2015-05-20
10,25414982,1634760123,"Out of Order (The Survivor's Club, #1)",3.85,"[{'author_id': '13850345', 'role': ''}]",Harmony Ink Press,"Corinna ""Corey"" Nguyen's life seems perfectly ...",[926363],US,180,2015-03-21


In [14]:
dataframes = [book_ya, book_romance, book_poetry, book_mistery, book_history, book_fantasy, book_children]

book_joined = pd.concat(dataframes, ignore_index=True)

book_joined.shape[0]

522438

In [15]:
book_joined.head()

,book_id,isbn,title,average_rating,authors,publisher,description,series,country_code,num_pages,publication_date
0,21401181,0698143760,"Half Bad (Half Life, #1)",3.80,"[{'author_id': '7314532', 'role': ''}]",Viking Children's,Wanted by no one.\nHunted by everyone.\nSixtee...,[493993],US,416,2014-03-04
1,22642971,0990662616,The Body Electric,3.71,"[{'author_id': '4018722', 'role': ''}]",Scripturient Books,The future world is at peace.\nElla Shepherd h...,[],US,351,2014-10-06
2,31556136,0062373374,Like Water,3.89,"[{'author_id': '6537142', 'role': ''}]",Balzer + Bray,A gorgeously written and deeply felt literary ...,[],US,304,2017-10-17
3,25421507,1941103863,The Ticket,3.82,"[{'author_id': '539783', 'role': ''}]",Firefly Southern Fiction,Librarian Note: See Alternate Cover Edition ....,[],US,210,2015-05-20
4,25414982,1634760123,"Out of Order (The Survivor's Club, #1)",3.85,"[{'author_id': '13850345', 'role': ''}]",Harmony Ink Press,"Corinna ""Corey"" Nguyen's life seems perfectly ...",[926363],US,180,2015-03-21


In [17]:
book_joined = book_joined.drop_duplicates(subset='book_id', keep='first')
book_joined.shape[0]

452982

In [20]:
authors_df = pd.read_json('/content/drive/MyDrive/Recommender Systems/GroupProject/Books/goodreads_book_authors.json', lines=True)
authors_df.head()

,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740


In [21]:
expanded_authors = book_joined['authors'].apply(pd.Series).stack().reset_index(level=1, drop=True).to_frame('author_info')

expanded_authors = expanded_authors.join(book_joined.drop('authors', axis=1), how='left')

expanded_authors['author_id'] = expanded_authors['author_info'].apply(lambda x: x.get('author_id'))

In [23]:
expanded_authors['author_id'] = expanded_authors['author_id'].astype(str)
authors_df['author_id'] = authors_df['author_id'].astype(str)

expanded_authors = expanded_authors.merge(authors_df[['author_id', 'name']], on='author_id', how='left')

expanded_authors['author_info'] = expanded_authors.apply(lambda row: {**row['author_info'], 'name': row['name']}, axis=1)

In [24]:
reconstructed_authors = expanded_authors.groupby('book_id')['author_info'].apply(list).reset_index()

book_joined = book_joined.merge(reconstructed_authors, on='book_id', how='left')

In [26]:
book_joined.drop('authors', axis=1, inplace=True)

book_joined.rename(columns={'author_info': 'authors'}, inplace=True)

book_joined.head()

,book_id,isbn,title,average_rating,publisher,description,series,country_code,num_pages,publication_date,authors
0,21401181,0698143760,"Half Bad (Half Life, #1)",3.80,Viking Children's,Wanted by no one.\nHunted by everyone.\nSixtee...,[493993],US,416,2014-03-04,"[{'author_id': '7314532', 'role': '', 'name': ..."
1,22642971,0990662616,The Body Electric,3.71,Scripturient Books,The future world is at peace.\nElla Shepherd h...,[],US,351,2014-10-06,"[{'author_id': '4018722', 'role': '', 'name': ..."
2,31556136,0062373374,Like Water,3.89,Balzer + Bray,A gorgeously written and deeply felt literary ...,[],US,304,2017-10-17,"[{'author_id': '6537142', 'role': '', 'name': ..."
3,25421507,1941103863,The Ticket,3.82,Firefly Southern Fiction,Librarian Note: See Alternate Cover Edition ....,[],US,210,2015-05-20,"[{'author_id': '539783', 'role': '', 'name': '..."
4,25414982,1634760123,"Out of Order (The Survivor's Club, #1)",3.85,Harmony Ink Press,"Corinna ""Corey"" Nguyen's life seems perfectly ...",[926363],US,180,2015-03-21,"[{'author_id': '13850345', 'role': '', 'name':..."


In [27]:
multiple_authors_case = book_joined[book_joined['authors'].apply(lambda x: len(x) > 1)]

if not multiple_authors_case.empty:
    display_case = multiple_authors_case.iloc[0]
    print("Authors:", display_case['authors'])

Authors: [{'author_id': '2905297', 'role': '', 'name': 'Lauren Kate'}, {'author_id': '1144378', 'role': '', 'name': 'Justine Eyre'}]


In [28]:
genres_df = pd.read_json('/content/drive/MyDrive/Recommender Systems/GroupProject/Books/goodreads_book_genres_initial.json', lines=True)
genres_df.head()

,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


In [29]:
book_joined['book_id'] = book_joined['book_id'].astype(str)
genres_df['book_id'] = genres_df['book_id'].astype(str)

book_joined = book_joined.merge(genres_df[['book_id', 'genres']], on='book_id', how='left')

In [34]:
books_df = book_joined

output_csv_books = '/content/drive/MyDrive/Recommender Systems/GroupProject/Books/CleanedData/books_df.csv'

books_df.to_csv(output_csv_books, index=False)

In [44]:
books_df.shape[0]

452982

# Clean Review Data

In [65]:
review_df = pd.read_json('/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/goodreads_reviews_poetry.json', lines=True)

In [66]:
review_df.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,3ca7375dba942a760e53b726c472a7dd,402128,28423ff309bc896c071a8d9df4a10e8a,5,I have three younger siblings and we grew up w...,Tue Jun 12 08:59:04 -0700 2012,Fri Jun 15 11:41:12 -0700 2012,,,0,0
1,0ef32090550901ead25cb0ea21c4d36b,92270,2db1180992e2b0b1631a3ac5644bde84,5,This is my favorite collection of poetry.,Mon Apr 14 18:42:40 -0700 2014,Mon Apr 14 18:43:05 -0700 2014,Wed Jan 01 00:00:00 -0800 1997,,0,0
2,0ef32090550901ead25cb0ea21c4d36b,908708,bca57fa40e92c9261b00b03dbebd96fe,4,"He's so disturbing. So very, very disturbing.",Tue Apr 22 13:58:10 -0700 2008,Tue Apr 22 13:58:33 -0700 2008,,,0,0
3,d37b46b2190ed7c518259f29b47a9b36,253264,cb1ebc02d8b2aff15735d513877463ce,5,I just reread this play for a class I am takin...,Wed Sep 27 19:08:08 -0700 2017,Sat Sep 30 06:39:45 -0700 2017,Wed Sep 27 00:00:00 -0700 2017,Tue Sep 26 00:00:00 -0700 2017,1,0
4,af157d0205b8a901dee6d4a2aed7e6ad,70885,8dca128b8e869048a7442c18659dbece,5,"Cuanto mas leo, mas me gusta. Su poesia es env...",Thu Jun 18 20:00:03 -0700 2015,Thu Jun 18 20:01:29 -0700 2015,Thu Jun 18 00:00:00 -0700 2015,Tue Jun 16 00:00:00 -0700 2015,0,0


In [67]:
books_df['book_id'] = books_df['book_id'].astype(str)

In [73]:
def clean_review_df(review_df_path):
    review_df = pd.read_json(review_df_path, lines=True)

    review_df.dropna(subset=['rating', 'review_text', 'date_added'], inplace=True)

    review_df = review_df[(review_df['review_text'] != '') & (review_df['date_added'] != '')]

    review_df.drop(['date_updated', 'n_comments', 'started_at'], axis=1, inplace=True)

    review_df['book_id'] = review_df['book_id'].astype(str)

    review_df = review_df[review_df['book_id'].isin(books_df['book_id'])]

    return review_df

In [74]:
review_ya_output_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/CleanedData/review_ya.csv'
review_ya_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/goodreads_reviews_young_adult.json'
review_ya = clean_review_df(review_ya_path)
review_ya.to_csv(review_ya_output_path, index=False)
review_ya.head()

,user_id,book_id,review_id,rating,review_text,date_added,read_at,n_votes
0,8842281e1d1347389f2ab93d60773d4d,2767052,248c011811e945eca861b5c31a549291,5,I cracked and finally picked this up. Very enj...,Wed Jan 13 13:38:25 -0800 2010,Sun Mar 25 00:00:00 -0700 2012,24
2,f8a89075dc6de14857561522e729f82c,18053080,785c8db878f4009da9741dea51f641da,4,Though the book started out slow and only star...,Sat Jan 11 17:58:41 -0800 2014,Sat Apr 12 00:00:00 -0700 2014,0
3,f8a89075dc6de14857561522e729f82c,17383543,34dc3c45d07e82718b05e73167259aef,2,"*Update - 10/27/13* - After some sleep, I thin...",Sun Apr 21 19:42:28 -0700 2013,Sat Oct 26 00:00:00 -0700 2013,0
4,f8a89075dc6de14857561522e729f82c,16651458,d8d6b590780256fef7ae4a9550fe3e0d,5,"This is a moving, heartbreaking, view into a l...",Fri Jan 11 11:42:42 -0800 2013,Mon Jan 14 00:00:00 -0800 2013,0
5,f8a89075dc6de14857561522e729f82c,10782699,972ce1267de0213e3032c685386890e6,5,I never thought I would enjoy a zombie books w...,Tue Oct 30 21:32:34 -0700 2012,Thu May 01 00:00:00 -0700 2014,0


In [75]:
review_poetry_output_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/CleanedData/review_poetry.csv'
review_poetry_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/goodreads_reviews_poetry.json'
review_poetry = clean_review_df(review_poetry_path)
review_poetry.to_csv(review_poetry_output_path, index=False)
review_poetry.head()

,user_id,book_id,review_id,rating,review_text,date_added,read_at,n_votes
0,3ca7375dba942a760e53b726c472a7dd,402128,28423ff309bc896c071a8d9df4a10e8a,5,I have three younger siblings and we grew up w...,Tue Jun 12 08:59:04 -0700 2012,,0
1,0ef32090550901ead25cb0ea21c4d36b,92270,2db1180992e2b0b1631a3ac5644bde84,5,This is my favorite collection of poetry.,Mon Apr 14 18:42:40 -0700 2014,Wed Jan 01 00:00:00 -0800 1997,0
2,0ef32090550901ead25cb0ea21c4d36b,908708,bca57fa40e92c9261b00b03dbebd96fe,4,"He's so disturbing. So very, very disturbing.",Tue Apr 22 13:58:10 -0700 2008,,0
3,d37b46b2190ed7c518259f29b47a9b36,253264,cb1ebc02d8b2aff15735d513877463ce,5,I just reread this play for a class I am takin...,Wed Sep 27 19:08:08 -0700 2017,Wed Sep 27 00:00:00 -0700 2017,1
4,af157d0205b8a901dee6d4a2aed7e6ad,70885,8dca128b8e869048a7442c18659dbece,5,"Cuanto mas leo, mas me gusta. Su poesia es env...",Thu Jun 18 20:00:03 -0700 2015,Thu Jun 18 00:00:00 -0700 2015,0


In [78]:
review_history_output_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/CleanedData/review_history.csv'
review_history_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/goodreads_reviews_history_biography.json'
review_history = clean_review_df(review_history_path)
review_history.to_csv(review_history_output_path, index=False)
review_history.head()

,user_id,book_id,review_id,rating,review_text,date_added,read_at,n_votes
2,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,,5
5,8842281e1d1347389f2ab93d60773d4d,22551730,754710070f38e1c59a45f6a85473b836,4,Another hard to put down nonfiction book from ...,Thu May 07 13:59:32 -0700 2015,Sat Jul 11 00:00:00 -0700 2015,20
6,8842281e1d1347389f2ab93d60773d4d,137554,6f74c6ca4ac2be7c71237d4d0fd4623d,0,Sacca and Nate recommend,Thu Mar 20 10:12:34 -0700 2014,,4
7,8842281e1d1347389f2ab93d60773d4d,5064,da4757c50c0edb081ca124d33db8186b,5,"I tore through The Pillars of the Earth, and c...",Sun Mar 02 12:34:19 -0800 2014,Sun Apr 27 00:00:00 -0700 2014,2
9,8842281e1d1347389f2ab93d60773d4d,40955,299706d01666058b1fb2a96b29a1260b,5,A truly inspirational book by a truly inspirat...,Sun Nov 18 16:31:28 -0800 2012,Fri Apr 17 00:00:00 -0700 2015,5


In [79]:
review_children_output_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/CleanedData/review_children.csv'
review_children_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/goodreads_reviews_children.json'
review_children = clean_review_df(review_children_path)
review_children.to_csv(review_children_output_path, index=False)
review_children.head()

,user_id,book_id,review_id,rating,review_text,date_added,read_at,n_votes
0,8842281e1d1347389f2ab93d60773d4d,23310161,f4b4b050f4be00e9283c92a814af2670,4,Fun sequel to the original.,Tue Nov 17 11:37:35 -0800 2015,,7
1,8842281e1d1347389f2ab93d60773d4d,17290220,22d424a2b0057b18fb6ecf017af7be92,5,One of my favorite books to read to my 5 year ...,Sat Nov 08 08:54:03 -0800 2014,Tue Jan 24 00:00:00 -0800 2017,4
2,8842281e1d1347389f2ab93d60773d4d,6954929,50ed4431c451d5677d98dd25ca8ec106,5,One of the best and most imaginative childrens...,Thu Oct 23 13:46:20 -0700 2014,,6
3,8842281e1d1347389f2ab93d60773d4d,460548,1e4de11dd4fa4b7ffa59b6c69a6b28e9,5,My daughter is loving this. Published in the 6...,Mon Dec 02 10:43:59 -0800 2013,,5
4,8842281e1d1347389f2ab93d60773d4d,11474551,2065145714bf747083a1c9ce81d5c4fe,5,A friend sent me this. Hilarious!,Wed May 11 22:38:11 -0700 2011,Wed May 11 00:00:00 -0700 2011,5


In [81]:
review_mystery_output_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/CleanedData/review_mystery.csv'
review_mystery_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/goodreads_reviews_mystery_thriller_crime.json'
review_mystery = clean_review_df(review_mystery_path)
review_mystery.to_csv(review_children_output_path, index=False)
review_mystery.head()

,user_id,book_id,review_id,rating,review_text,date_added,read_at,n_votes
2,8842281e1d1347389f2ab93d60773d4d,32283133,8e4d61801907e591018bdc3442a9cf2b,0,http://www.telegraph.co.uk/culture/10...,Tue Nov 01 11:09:18 -0700 2016,,9
3,8842281e1d1347389f2ab93d60773d4d,17860739,022bb6daffa49adc27f6b20b6ebeb37d,4,An amazing and unique creation: JJ Abrams and ...,Wed Mar 26 13:51:30 -0700 2014,Sun Sep 21 00:00:00 -0700 2014,7
5,8842281e1d1347389f2ab93d60773d4d,6652906,4276918357312212384ac6415ceb9159,3,** spoiler alert ** \n Hooked me equally as we...,Mon Mar 29 15:54:28 -0700 2010,Thu Apr 08 00:00:00 -0700 2010,1
6,8842281e1d1347389f2ab93d60773d4d,2429135,efee7a3f620ed640366226caa7b138f0,5,I haven't stayed up until 5am to finish a book...,Mon Mar 15 14:05:04 -0700 2010,Sun Mar 28 00:00:00 -0700 2010,13
8,8842281e1d1347389f2ab93d60773d4d,7661,fc5c2b74043cb5132b5b607c16a68211,3,"A quick read, but I enjoyed the focus on gener...",Wed Aug 12 16:34:06 -0700 2009,Sun Aug 09 00:00:00 -0700 2009,0


In [83]:
review_romance_output_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/CleanedData/review_romance.csv'
review_romance_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/goodreads_reviews_romance.json'
review_romance = clean_review_df(review_romance_path)
review_romance.to_csv(review_children_output_path, index=False)
review_romance.head()

,user_id,book_id,review_id,rating,review_text,date_added,read_at,n_votes
0,8842281e1d1347389f2ab93d60773d4d,1893,5347a776a1703b823ce029d68ae98275,5,** spoiler alert ** \n So the other day Elizab...,Tue Oct 10 19:08:05 -0700 2006,Sun Oct 01 00:00:00 -0700 2006,155
5,72fb0d0087d28c832f15776b0d936598,2718668,768e4a82b89e36055706857e9268b8e9,5,I really enjoyed this book! Chick lit with bra...,Wed Oct 30 11:13:28 -0700 2013,,0
7,7504b2aee1ecb5b2872d3da381c6c91e,15507958,63ff74279e46b247cb1754313b160006,4,I finished reading this days ago and cant get ...,Wed Sep 10 19:33:44 -0700 2014,Sun Sep 21 00:00:00 -0700 2014,0
10,012515e5802b2e0f42915118c90fa04b,9462815,a99a5a35d2592795791ebe3e49c57b99,2,Wow I lost it on the last page. Yay for happy ...,Sat Nov 08 08:59:14 -0800 2014,Mon Nov 10 12:46:36 -0800 2014,0
11,012515e5802b2e0f42915118c90fa04b,20572939,812febaa143162ed49152547ff7a3e08,5,There was sobbing. Quite a lot. \n The last fe...,Sun Sep 14 12:55:35 -0700 2014,Mon Sep 15 16:52:15 -0700 2014,0


In [82]:
review_fantasy_output_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/CleanedData/review_fantasy.csv'
review_fantasy_path = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/goodreads_reviews_fantasy_paranormal.json'
review_fantasy = clean_review_df(review_fantasy_path)
review_fantasy.to_csv(review_children_output_path, index=False)
review_fantasy.head()

,user_id,book_id,review_id,rating,review_text,date_added,read_at,n_votes
4,8842281e1d1347389f2ab93d60773d4d,13239822,a582bfa8efd69d453a5a21a678046b36,3,"This book has a great premise, and is full of ...",Mon Jul 02 16:04:16 -0700 2012,Wed Aug 15 00:00:00 -0700 2012,7
5,8842281e1d1347389f2ab93d60773d4d,62291,e23965adda7ce7e7e415a1339e169065,5,** spoiler alert ** \n Loved it. The epic saga...,Tue Jul 12 12:57:07 -0700 2011,Sat Aug 20 00:00:00 -0700 2011,5
6,8842281e1d1347389f2ab93d60773d4d,41804,2bcbe2300fe1801197f9bfe1f1c883ba,5,As an engineer I couldn't help but love this b...,Wed Mar 25 10:56:59 -0700 2009,Wed Apr 08 00:00:00 -0700 2009,12
7,8842281e1d1347389f2ab93d60773d4d,136251,132eab4c9a3724493204cc083e0e2ecc,5,Loved every minute. So sad there isn't another...,Mon Dec 22 10:38:27 -0800 2008,Sun Jan 02 00:00:00 -0800 2011,6
10,8842281e1d1347389f2ab93d60773d4d,34497,43828b41e8099f892cfee80a8244684a,3,"This should be 3.5 stars. I really enjoyed it,...",Tue Jun 03 16:42:44 -0700 2008,Mon Nov 10 00:00:00 -0800 2008,2


In [84]:
dataframes_review = [review_ya, review_romance, review_poetry, review_mystery, review_history, review_fantasy, review_children]

reviews_joined = pd.concat(dataframes_review, ignore_index=True)

In [85]:
reviews_joined.shape[0]

8211681

In [87]:
reviews_joined = reviews_joined.drop_duplicates(subset='review_id', keep='first')
reviews_joined.shape[0]

6555799

In [88]:
reviews_joined.head()

,user_id,book_id,review_id,rating,review_text,date_added,read_at,n_votes
0,8842281e1d1347389f2ab93d60773d4d,2767052,248c011811e945eca861b5c31a549291,5,I cracked and finally picked this up. Very enj...,Wed Jan 13 13:38:25 -0800 2010,Sun Mar 25 00:00:00 -0700 2012,24
1,f8a89075dc6de14857561522e729f82c,18053080,785c8db878f4009da9741dea51f641da,4,Though the book started out slow and only star...,Sat Jan 11 17:58:41 -0800 2014,Sat Apr 12 00:00:00 -0700 2014,0
2,f8a89075dc6de14857561522e729f82c,17383543,34dc3c45d07e82718b05e73167259aef,2,"*Update - 10/27/13* - After some sleep, I thin...",Sun Apr 21 19:42:28 -0700 2013,Sat Oct 26 00:00:00 -0700 2013,0
3,f8a89075dc6de14857561522e729f82c,16651458,d8d6b590780256fef7ae4a9550fe3e0d,5,"This is a moving, heartbreaking, view into a l...",Fri Jan 11 11:42:42 -0800 2013,Mon Jan 14 00:00:00 -0800 2013,0
4,f8a89075dc6de14857561522e729f82c,10782699,972ce1267de0213e3032c685386890e6,5,I never thought I would enjoy a zombie books w...,Tue Oct 30 21:32:34 -0700 2012,Thu May 01 00:00:00 -0700 2014,0


In [91]:
reviews_df = reviews_joined

output_csv_reviews = '/content/drive/MyDrive/Recommender Systems/GroupProject/Reviews/CleanedData/reviews_df.csv'

reviews_df.to_csv(output_csv_reviews, index=False)

# Create User Dataframe

In [93]:
user_reviews_count = reviews_df.groupby('user_id')['review_id'].count().reset_index(name='number_of_reviews')

In [94]:
users_df = user_reviews_count

In [95]:
print(users_df.head())

                            user_id  number_of_reviews
0  00000377eea48021d3002730d56aca9a                  3
1  00009e46d18f223a82b22da38586b605                 54
2  0000c3d51aa099745e93a4e99c4856c8                 97
3  0001085188e302fc6b2568de45a5f56b                  2
4  00013f3df6711fc887013c9692de4416                  1


In [102]:
output_csv_users = '/content/drive/MyDrive/Recommender Systems/GroupProject/users_df.csv'

users_df.to_csv(output_csv_users, index=False)

In [98]:
users_df.shape[0]

359788

In [99]:
reviews_df.shape[0]

6555799

In [100]:
books_df.shape[0]

452982